In [2]:
import sys; sys.path.append('methods/')
from regridding import *

In [3]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [5]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("Westminster_Parliamentary_Constituencies_December_2019_Boundaries_UK_BUC")
sf

,pcon19cd,pcon19nm,bng_e,bng_n,long,lat,objectid,st_lengths,st_areasha,geometry
0,E14000530,Aldershot,484884,155126,-0.78410,51.288952,1,37241.703656,5.258835e+07,"POLYGON ((485406.902 159918.603, 486138.297 15..."
1,E14000531,Aldridge-Brownhills,404723,302568,-1.93166,52.620869,2,35258.924699,4.398713e+07,"POLYGON ((406519.098 305054.298, 405782.501 30..."
2,E14000532,Altrincham and Sale West,374132,389051,-2.39049,53.397659,3,40812.004306,5.087472e+07,"POLYGON ((379104.096 393143.903, 377127.904 39..."
3,E14000533,Amber Valley,440478,349675,-1.39770,53.042831,4,54421.854957,1.246901e+08,"POLYGON ((444868.402 353958.100, 444113.303 35..."
4,E14000534,Arundel and South Downs,510686,115542,-0.42635,50.928711,5,207778.021149,6.469497e+08,"POLYGON ((506641.497 128757.203, 506389.800 12..."
...,...,...,...,...,...,...,...,...,...,...
645,W07000076,Caerphilly,316860,188276,-3.20144,51.587181,646,63865.409578,1.150569e+08,"POLYGON ((315131.401 200637.498, 315308.299 19..."
646,W07000077,Islwyn,320517,196276,-3.15048,51.659618,647,59239.492373,1.117145e+08,"POLYGON ((322947.397 199706.204, 322974.804 19..."
647,W07000078,Vale of Glamorgan,302293,173081,-3.40742,51.448250,648,91342.706619,3.148419e+08,"POLYGON ((302260.803 179531.999, 302783.903 17..."
648,W07000079,Cardiff West,312437,178939,-3.26294,51.502571,649,36926.700752,5.081285e+07,"POLYGON ((312599.599 182852.798, 312692.900 18..."


In [6]:
filesnames=["maxtmax_UKCP18-12km_ens-mean-maps_19801201-20101130_annual.nc","maxtmax_UKCP18-12km_ens-mean-maps_20191201-20491130_annual.nc","maxtmax_UKCP18-12km_ens-mean-maps_20491201-20791130_annual.nc","ndhi_UKCP18-12km_ens-mean-maps_19801201-20101130_annual.nc","ndhi_UKCP18-12km_ens-mean-maps_20191201-20491130_annual.nc","ndhi_UKCP18-12km_ens-mean-maps_20491201-20791130_annual.nc","pr_UKCP18-12km_ens-mean-maps_19801201-20101130_DJF.nc","pr_UKCP18-12km_ens-mean-maps_19801201-20101130_JJA.nc","pr_UKCP18-12km_ens-mean-maps_20191201-20491130_DJF.nc","pr_UKCP18-12km_ens-mean-maps_20191201-20491130_JJA.nc","pr_UKCP18-12km_ens-mean-maps_20491201-20791130_DJF.nc","pr_UKCP18-12km_ens-mean-maps_20491201-20791130_JJA.nc","r99ptot_UKCP18-12km_ens-mean-maps_19801201-20101130_annual.nc","r99ptot_UKCP18-12km_ens-mean-maps_20191201-20491130_annual.nc","r99ptot_UKCP18-12km_ens-mean-maps_20491201-20791130_annual.nc","rx5day_UKCP18-12km_ens-mean-maps_19801201-20101130_annual.nc","rx5day_UKCP18-12km_ens-mean-maps_20191201-20491130_annual.nc","rx5day_UKCP18-12km_ens-mean-maps_20491201-20791130_annual.nc","spei6dd_UKCP18-12km_ens-mean-maps_19801201-20101130_annual.nc","spei6dd_UKCP18-12km_ens-mean-maps_20191201-20491130_annual.nc","spei6dd_UKCP18-12km_ens-mean-maps_20491201-20791130_annual.nc","tas_UKCP18-12km_ens-mean-maps_19801201-20101130_annual.nc","tas_UKCP18-12km_ens-mean-maps_20191201-20491130_annual.nc","tas_UKCP18-12km_ens-mean-maps_20491201-20791130_annual.nc"]
varnames_in=["maxtmax","maxtmax","maxtmax","ndhi","ndhi","ndhi","pr","pr","pr","pr","pr","pr","r99ptot","r99ptot","r99ptot","rx5day","rx5day","rx5day","spei6dd","spei6dd","spei6dd","tas","tas","tas"]
varnames_out=["maxtmax_baseline","maxtmax_2030s","maxtmax_2060s","ndhi_baseline","ndhi_2030s","ndhi_2060s","winter_rain_baseline","summer_rain_baseline","winter_rain_2030s","summer_rain_2030s","winter_rain_2060s","summer_rain_2060s","r99ptot_baseline","r99ptot_2030s","r99tpot_2060s","rx5day_baseline","rx5day_2030s","rx5day_2060s","spei6dd_baseline","spei6dd_2030s","spei6dd_2060s","tas_baseline","tas_2030s","tas_2060s"]


In [8]:
for i in range(24):
    fnm = "../gridded_data/"+filesnames[i]
    ds = xr.open_dataset(fnm)
    ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
    da=ds[varnames_in[i]].isel(ensemble=0, period=0, season=0)
    rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "objectid")
    # apply the regionmask to the data and average over the x & y dimensions
    region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])
    # find nearest neighbour (this takes the most time)
    region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.objectid[i]]).assign_coords(constituency = ("region", [sf.pcon19cd[i]])) for i in range(len(sf))], "region")
    # combine regionmask with nearest neighbour where regionmask didn't pick anything up
    region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")
    region_all=region_all.assign_coords(name=('region',sf.pcon19nm))
    # output the values into a csv file
    region_all.to_dataframe(name=varnames_out[i]).reset_index().to_csv(varnames_out[i]+'.csv')
    print("outputting ",varnames_out[i]+'.csv')




outputting  maxtmax_baseline.csv
outputting  maxtmax_2030s.csv
outputting  maxtmax_2060s.csv
outputting  ndhi_baseline.csv
outputting  ndhi_2030s.csv
outputting  ndhi_2060s.csv
outputting  winter_rain_baseline.csv
outputting  summer_rain_baseline.csv
outputting  winter_rain_2030s.csv
outputting  summer_rain_2030s.csv
outputting  winter_rain_2060s.csv
outputting  summer_rain_2060s.csv
outputting  r99pot_baseline.csv
outputting  r99pot_2030s.csv
outputting  r99pot_2060s.csv
outputting  rx5day_baseline.csv
outputting  rx5day_2030s.csv
outputting  rx5day_2060s.csv
outputting  spei6dd_baseline.csv
outputting  spei6dd_2030s.csv
outputting  spei6dd_2060s.csv
outputting  tas_baseline.csv
outputting  tas_2030s.csv
outputting  tas_2060s.csv


In [11]:
# Combine into a big dataframe
import pandas
all_files = os.listdir()    
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
for i in range(24):
    this_csv=pandas.read_csv(csv_files[i])
    if i == 0:
        csv_out=this_csv.iloc[:,[2,6,7]]
    else:
        var=csv_files[i].rsplit( ".", 1 )[ 0 ]
        csv_out[var]=this_csv.iloc[:,7]

csv_out.to_csv('../by_mp/climate.csv',index=False)


